In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/week4-practice-hackathon-2024/sample_submission.csv
/kaggle/input/week4-practice-hackathon-2024/Train.csv
/kaggle/input/week4-practice-hackathon-2024/Test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Train.csv")

In [3]:
print(train_df.shape)

(40776, 32)


In [4]:
train_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V45,V47,V50,V51,V52,V59,V63,V72,V75,class
0,0.0,1.0,243.0,29,4528.0,138.0,3.0,29,201.0,0.0,...,2,3,71.0,0,0,46.0,1,2,0,1
1,0.0,20.0,8.0,14,4183.0,183.0,4.0,26,156.0,0.0,...,5,7,20.0,0,2,13.0,9,6,0,1
2,14.0,2.0,2.0,9,1113.0,15.0,37.0,31,78.0,1.0,...,7,5,11.0,0,0,7.0,9,2,1,0
3,0.0,175.0,17.0,16,4250.0,68.0,8.0,30,152.0,0.0,...,1,3,0.0,1,1,31.0,6,4,5,1
4,2.0,-1.0,1.0,3,0.0,0.0,2.0,5,6.0,1.0,...,1,5,0.0,0,0,2.0,8,0,1,1


In [5]:
train_df.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V13', 'V19', 'V22', 'V30', 'V33', 'V35', 'V36', 'V40', 'V41', 'V42',
       'V43', 'V45', 'V47', 'V50', 'V51', 'V52', 'V59', 'V63', 'V72', 'V75',
       'class'],
      dtype='object')

In [6]:
X = train_df.drop(['class'], axis=1)
y = train_df['class']

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3)

In [8]:
print(y_train.value_counts())

class
0    14304
1    14239
Name: count, dtype: int64


In [9]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Random Forest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)


RandomForestClassifier()

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV
# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': [ 'sqrt'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='f1')
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_



Fitting 3 folds for each of 81 candidates, totalling 243 fits


In [11]:
y_pred = best_rf.predict(X_val)
f1 = f1_score(y_val, y_pred)


In [12]:
print(f"Tuned Random Forest F1 score: {f1}")


Tuned Random Forest F1 score: 0.6749628238240588


In [13]:
test_data = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Test.csv")

In [14]:
test_data.shape

(17476, 32)

In [15]:
Index=test_data['Index']
test_data.drop(['Index'],axis=1,inplace=True)

In [16]:
y_test = best_rf.predict(test_data)

In [17]:
y_test

array([0, 0, 1, ..., 1, 0, 0])

In [18]:
result = pd.DataFrame({
    'Index': Index,
    'class': y_test
})

In [19]:
result

,Index,class
0,40977,0
1,24157,0
2,56238,1
3,55411,1
4,53175,0
...,...,...
17471,17197,1
17472,14094,0
17473,34598,1
17474,17845,0


In [20]:
result.to_csv("submission5.csv", index=False) #(for making submission file)